<div align="right">Python 3 [conda default]<div>

# Towers of Hanoi

![Towers of Hanoi](https://upload.wikimedia.org/wikipedia/commons/6/60/Tower_of_Hanoi_4.gif)

The "Towers of Hanoi' problem is a popular choice by computer programming training classes.  An execellent write-up of it can be found here:

[Python Course.eu - Towers of Hanoi](http://www.python-course.eu/towers_of_hanoi.php)

Wikipedia also has some great information about the problem, its history, and related programming concerns:  [Hannoi on Wikipedia](http://en.wikipedia.org/wiki/Tower_of_Hanoi) (though some of it gets highly technical).

## In this Notebook
- [The Solution](#solution):  Immediately below is a solution to the problem organized as OO Python code.  This code leverages concepts and ideas from the best of what is found in the research section, but creates a unique implementation that could be used to achieve multiple objectives: output the answer, store the answer, tell us different things about the answer.  This code  also illustrates many concepts of the Python programming language that students and non-experts may find useful.<br/><br/>

- [OO Design Considerations For The Solution](#ooDesign) - notes on the object design heirarchy (what was chosen over what was rejected).


- [Putting a Tracer on The Solution To Watch Recursion in Action](#trace) - This section is an experiment purely for the educational value.  It makes it possible to watch recursive function calls traced through the hanoi solution object.


- [Related Research and Experiments](#Research):  This section contains code from multiple sources showing approaches to the Towers of Hanoi problem.  It also contains edits to this code that help unmask things about the algorithms' inner workings, as well as enhancements and experiments that ultimately pave the way for the final solution given at the start of this notebook.

## Version Notes
This code was originally written in Python 2.7.  It was later converted to Python 3.6.  The two changes that were required in order to do this were: 
- import sys (was not needed under Python 2.7
- .pop() worked on range objects in Python 2.7, they had to be wrapped in list() to work under Python 3.6
  - Example: `(list(range(numDisks, 0, -1)), 1)`
- all the rest of this code is unchanged from the original Python 2.7 experiment

<a id="solution" name="solution"></a>
## An Object Oriented Solution to The Towers of Hanoi

In [1]:
''' 
    This solution leverages the best of the code in this notebook to attempt to create something
    extensible, self-contained, and capable of delivering different outputs to meet different needs. 
    It gets longer than the more elegant solutions in the "Research" section, but the design
    wraps the basic functionality with different features that take into account different potential
    future use cases.

'''
### Verson Two:  Object code

import pandas as pd
from warnings import warn
from warnings import filterwarnings
import numpy as np
import sys   ## added during PY 2.7 to 3.6 upgrade test

class SimpleWarning(object):
    '''class SimpleWarning() -->\n\n configures warn() for the most common "alert the user" use case. '''
    def __init__(self, warnText, wStackLevel=1, wCategory=RuntimeWarning):
        self._wrnTxt = "\n%s" %(warnText)
        filterwarnings("once")
        warn(self._wrnTxt, stacklevel=wStackLevel, category=wCategory)
        sys.stderr.flush()  # this provides warning ahead of the output instead of after it
                            # sys is imported by warnings so we don't have to import it here
                            # common categories to use: UserWarning, Warning, RuntimeWarning, ResourceWarning
                
    def reInitialize(self, riWarnText, riWStackLevel=1, riWCategory=RuntimeWarning):
        '''SimpleWarning.reInitialize(...)-->\n\nFor multiple warnings in one code procedure, 
this function can reinitialize the same object to be reused.'''
        self.__init__(self, riWarnText, riWStackLevel, riWCategory)
        
class HanoiSolution(object):
    _mvListValues = ["Step", "Count", "None", "Visual"]
    
    def __init__(self, numDisks, moveList="Visual", divider=30):        
        # conditions for warning and to help control all output that comes later:
        if numDisks <= 0:
            self._tmpTxt = "is not valid for the number of disks.  Resetting number of disks to default."
            self._tmpTxt = "%s %s" %(numDisks, self._tmpTxt) 
            # warn("\n%s %s" %(numDisks, self._tmpTxt))
            self._hsWarn = SimpleWarning(self._tmpTxt)
            numDisks = 3

        if numDisks > 1:
            self._chr1 = 's'   # used to ensure printed output is plural if disks > 1
        else:                  # set plurality conditions here and then just add self._chr1
            self._chr1 = ''    # instead of 's' on words throughout the code where it applies
            
        if numDisks > 9:       # for 10 disks or more (issue a warning)
            self._tmpTxt =  "disks selected.  The number of steps in a solution grows at an accelerated rate"
            self._tmpTxt += " as the number of disks increases. \nThis program may take a while to complete.\n"
            self._tmpTxt += "Please be patient..."
            self._tmpTxt = "%d %s" %(numDisks, self._tmpTxt) 
            # warn("\n%s %s" %(numDisks, self._tmpTxt))
            self._hsWarn = SimpleWarning(self._tmpTxt)
        
        # peg structure: ( [ disks ], Peg_ID_Number )
        self._peg1 = (list(range(numDisks, 0, -1)), 1)
        self._peg2 = ([], 2)
        self._peg3 = ([], 3)
        
        self.dsks = numDisks             # number of disks for simulation
        self.moveCount = 0               # move counter
        self.divChars = divider          # number of characters for divider used in output   
        self.moveListDefault = moveList  # what type of output from the moveList do you want?
                                         # store the answer as a default for the object to use
                        
        # invalid moveList argument is reset to default and a warning is output:
        if moveList not in self._mvListValues:
            self._tmpTxt = "is not a valid moveList arg for _output_diskProgress(...).  " + \
                   "Default will be used."
            self._tmpTxt = "'%s' %s" %(moveList, self._tmpTxt) 
            # warn("\n'%s' %s" %(moveList, self._tmpTxt))
            self._hsWarn = SimpleWarning(self._tmpTxt)
            self.moveListDefault = self._mvListValues[-1]  # last value, by convention is default for obj class
                                                           # make it default for this instance of the class
        else:
            self.moveListDefault = moveList

        self._moveDisks(nDisks=numDisks, source=self._peg1, target=self._peg3, 
                        auxiliary=self._peg2, moveList=self.moveListDefault)
        
        if moveList != "None":  print(self.__str__()) # this outputs final answer with move count
                                                      # at end of all moveList args that include printed
                                                      # output
    
    # meat and potatoes of the algorithm:  sumulation of moving the disks from one peg to another
    def _moveDisks(self, nDisks, source, target, auxiliary, moveList):        
        if nDisks > 0:           
            # move n-1 disks from source to auxiliary
            self._moveDisks(nDisks-1, source, auxiliary, target, moveList)
            if self.moveCount == 0:
                # output initial state if appropriate
                self._diskMovementProgression(nDisks, source, target, moveList)

            self.moveCount += 1  # increment counter of how many steps it takes
            
            # move the nth disk from source to target
            target[0].append(source[0].pop())
            
            # in this object:  outputs the moves in accordance with moveList argument
            self._diskMovementProgression(nDisks, source, target, moveList)

            # move the n-1 disks that were left on auxiliary to target
            self._moveDisks(nDisks-1, auxiliary, target, source, moveList)
    
    def _diskMovementProgression(self, nDisks, source, target, moveList):
        # this function sets up ability to over-ride the function call in the middle 
        # of moveDisk by child objects
        self._output_diskProgress(nDisks, source, target, moveList)
    
    def _output_diskProgress(self, nDisks, source, target, moveList):
        # Display our progress (create each step of the answer and output it)
        if moveList == "Visual" or moveList == "Step":
            if moveList == "Visual":
                if self.moveCount > 0: 
                    print("Step %d:" %self.moveCount)
                else:             # in this context, moveCount = 0
                    print("Initial State:")
            
            # used by both "Visual" and "Step"
            if self.moveCount == 0:
                pass
            else:
                print("Move disk " + str(nDisks) + " from peg " + str(source[1]) + 
                      " to peg " + str(target[1]))
            
            if moveList == "Visual":
                print("-"*self.divChars)
                print(str(self._peg1[0]) + '\n' + str(self._peg2[0]) + '\n' + str(self._peg3[0]) + 
                      '\n' + '#'*self.divChars)
        elif moveList == "None" or moveList == "Count":
            pass
        else:
            # this scenario should never occur the way this code is written.
            # if it does, we want the code to throw an error so we know to look into it
            raise ValueError("%s is not a valid arg for moveList in _output_diskProgress(...).")
            
    def reInitialize(self, nDisks, moveList, divider=30):
        # allows resetting the object for a new simulation without having to create a new instance
        self.__init__(nDisks, moveList, divider)
                
    def __str__(self):
        # what we want to see for print(HanoiSolution)
        return ("%d disk" + self._chr1 + " would take %d move" + self._chr1 +
               " to solve.") %(self.dsks, self.moveCount)

class HanoiStoredSolution(HanoiSolution):
    dfCellDataType = np.int64
    def __init__(self, numDisks, moveList="Stored", divider=30):
        self._solutionDF = pd.DataFrame({'disk':[],'fromPeg':[], 'toPeg':[]}, dtype=self.dfCellDataType)        
        self._mvListValues.append("Stored")
        super(HanoiStoredSolution, self).__init__(numDisks, moveList, divider)
        # alternatively, this should also work: HanoiSolution.__init__(self, numDisks, moveList, divider)
        
        if moveList == "Stored":
            # print("You selected to store the movelist with this agrument: %s" %moveList) # debug statement
            print("The move list is stored in a dataframe accessible with `.get_solutionDF()`:")
            print(self.get_solutionDF())

    def _store_diskProgress(self, dsk, sourceID, targetID):
        # Builds this: self._solutionPD = pd.DataFrame({ 'disk':[],'fromPeg':[], 'toPeg':[] })
        if self.moveCount > 0:
            self._solutionDF = self._solutionDF.append(pd.DataFrame({ 'disk':[dsk], 
                                                       'fromPeg':[sourceID],'toPeg':[targetID] }), 
                                                       ignore_index=True)
            
    def _diskMovementProgression(self, nDisks, source, target, moveList="Visual"):
        # tell us about it and store the results:
        if moveList == "Stored":
            output_moveList = "None"
        else:
            output_moveList = moveList
            
        self._output_diskProgress(nDisks, source, target, output_moveList)
        self._store_diskProgress(nDisks, source[1], target[1])
        
    def get_solutionDF(self):
        return self._solutionDF
    
print("Hanoi Solution Objects Loaded and ready to use.")

Hanoi Solution Objects Loaded and ready to use.


In [2]:
# doc strings for SimpleWarning
print(SimpleWarning.__doc__)
print("-"*72)
print(SimpleWarning.reInitialize.__doc__)

class SimpleWarning() -->

 configures warn() for the most common "alert the user" use case. 
------------------------------------------------------------------------
SimpleWarning.reInitialize(...)-->

For multiple warnings in one code procedure, 
this function can reinitialize the same object to be reused.


### Tests of Hanoi Solution Objects
These tests are designed to test and show all the functionality built into the Hanoi solution objects.  Comments in each cell indicate what is being tested

In [3]:
# Method Resolution Order for the class objects
print(HanoiSolution.__mro__)
print(HanoiStoredSolution.__mro__) 

(<class '__main__.HanoiSolution'>, <class 'object'>)
(<class '__main__.HanoiStoredSolution'>, <class '__main__.HanoiSolution'>, <class 'object'>)


In [4]:
# pass in an invalid moveList argument ... show the warning that is displayed but code continues to execute
import sys
myHanoiTower = HanoiSolution(1, "Something Stupid")

/Users/mitchmac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: 
'Something Stupid' is not a valid moveList arg for _output_diskProgress(...).  Default will be used.


Initial State:
------------------------------
[1]
[]
[]
##############################
Step 1:
Move disk 1 from peg 1 to peg 3
------------------------------
[]
[]
[1]
##############################
1 disk would take 1 move to solve.


In [5]:
# pass in invalid numDisks argument ... show warning, code executes with object default
# note:  warning comes at the end of the output
myHanoiTower = HanoiSolution(0)  # default moveList = 'Visible'

/Users/mitchmac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: 
0 is not valid for the number of disks.  Resetting number of disks to default.


Initial State:
------------------------------
[3, 2, 1]
[]
[]
##############################
Step 1:
Move disk 1 from peg 1 to peg 3
------------------------------
[3, 2]
[]
[1]
##############################
Step 2:
Move disk 2 from peg 1 to peg 2
------------------------------
[3]
[2]
[1]
##############################
Step 3:
Move disk 1 from peg 3 to peg 2
------------------------------
[3]
[2, 1]
[]
##############################
Step 4:
Move disk 3 from peg 1 to peg 3
------------------------------
[]
[2, 1]
[3]
##############################
Step 5:
Move disk 1 from peg 2 to peg 1
------------------------------
[1]
[2]
[3]
##############################
Step 6:
Move disk 2 from peg 2 to peg 3
------------------------------
[1]
[]
[3, 2]
##############################
Step 7:
Move disk 1 from peg 1 to peg 3
------------------------------
[]
[]
[3, 2, 1]
##############################
3 disks would take 7 moves to solve.


In [6]:
# reinitialize existing object with new number of disks
# just output the final count
myHanoiTower.reInitialize(25, "Count")  # this took maybe 3 minutes to run on my computer

/Users/mitchmac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: 
25 disks selected.  The number of steps in a solution grows at an accelerated rate as the number of disks increases. 
This program may take a while to complete.
Please be patient...


25 disks would take 33554431 moves to solve.


In [7]:
anotherHanoiTower = HanoiSolution(13, "None")  # warning kicks in if numDisks is >= 10
                                               # this cell is part of testing the "None" option for moveList

/Users/mitchmac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: 
13 disks selected.  The number of steps in a solution grows at an accelerated rate as the number of disks increases. 
This program may take a while to complete.
Please be patient...


In [8]:
print(anotherHanoiTower)     # now we can ask it for the answer

13 disks would take 8191 moves to solve.


In [9]:
anotherHanoiTower.moveCount  # or obtain the final answer to send to other code

8191

In [10]:
# reinitialize with "Step" output  ... using the same object but reinitializing for new number of Disks and Output request
myHanoiTower.reInitialize(7, "Step")

Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 3 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 4 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 2 from peg 3 to peg 1
Move disk 1 from peg 2 to peg 1
Move disk 3 from peg 3 to peg 2
Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 5 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 3 from peg 2 to peg 1
Move disk 1 from peg 3 to peg 2
Move disk 2 from peg 3 to peg 1
Move disk 1 from peg 2 to peg 1
Move disk 4 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 3 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move dis

In [11]:
# object stores these elements once created:
print(myHanoiTower.divChars)
print(myHanoiTower.dsks)
print(myHanoiTower.moveCount)
print(myHanoiTower.moveListDefault)    # build a describe or summary function for this later?

30
7
127
Step


In [12]:
myHanoiSSTower = HanoiStoredSolution(3)

3 disks would take 7 moves to solve.
The move list is stored in a dataframe accessible with `.get_solutionDF()`:
   disk  fromPeg  toPeg
0     1        1      3
1     2        1      2
2     1        3      2
3     3        1      3
4     1        2      1
5     2        2      3
6     1        1      3


In [13]:
# access this element and reset it if your computer is not 64bit:
myHanoiSSTower.dfCellDataType

numpy.int64

In [14]:
myHanoiSSTower = HanoiStoredSolution(3, "Count")  # output stops with the move count for the solution
myHanoiSSTower.get_solutionDF()                   # produces the df table if last line on Jupyter

3 disks would take 7 moves to solve.


,disk,fromPeg,toPeg
0,1,1,3
1,2,1,2
2,1,3,2
3,3,1,3
4,1,2,1
5,2,2,3
6,1,1,3


In [15]:
myHanoiSSTower = HanoiStoredSolution(3, "None")   # outputs Nothing (except initialization lines)
myHanoiSSTower.get_solutionDF()                   # produces the df table if last line on Jupyter
                                                  # in production code, might turn off class object intialization lines

,disk,fromPeg,toPeg
0,1,1,3
1,2,1,2
2,1,3,2
3,3,1,3
4,1,2,1
5,2,2,3
6,1,1,3


In [16]:
# other stored elements in the object:
print(myHanoiSSTower.divChars)
print(myHanoiSSTower.dsks)
print(myHanoiSSTower.moveCount)
print(myHanoiSSTower.moveListDefault)

30
3
7
None


In [17]:
myHanoiSSTower.reInitialize(10, "Count") # just showing some more of the parent code working in the child object
# myHanoiSSTower.get_solutionDF()        # solution DF is big, uncomment this line to view it

/Users/mitchmac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: 
10 disks selected.  The number of steps in a solution grows at an accelerated rate as the number of disks increases. 
This program may take a while to complete.
Please be patient...


10 disks would take 1023 moves to solve.


In [18]:
myHanoiSSTower.get_solutionDF().tail()  # this validates the count is right by showing final records in the DF
                                        # note: index runs 0 ... 1022, so 1023 is the correct count

,disk,fromPeg,toPeg
1018,1,3,1
1019,3,2,3
1020,1,1,2
1021,2,1,3
1022,1,2,3


In [19]:
myHanoiSSTower2 = HanoiStoredSolution(5, "Visual", 72)  # args: numDisks, moveList (type), divider (num chars)

Initial State:
------------------------------------------------------------------------
[5, 4, 3, 2, 1]
[]
[]
########################################################################
Step 1:
Move disk 1 from peg 1 to peg 3
------------------------------------------------------------------------
[5, 4, 3, 2]
[]
[1]
########################################################################
Step 2:
Move disk 2 from peg 1 to peg 2
------------------------------------------------------------------------
[5, 4, 3]
[2]
[1]
########################################################################
Step 3:
Move disk 1 from peg 3 to peg 2
------------------------------------------------------------------------
[5, 4, 3]
[2, 1]
[]
########################################################################
Step 4:
Move disk 3 from peg 1 to peg 3
------------------------------------------------------------------------
[5, 4]
[2, 1]
[3]
######################################################################

In [20]:
myHanoiSSTower2.get_solutionDF()  # show stored DF in the object

,disk,fromPeg,toPeg
0,1,1,3
1,2,1,2
2,1,3,2
3,3,1,3
4,1,2,1
5,2,2,3
6,1,1,3
7,4,1,2
8,1,3,2
9,2,3,1


In [21]:
myHanoiSSTower2.reInitialize(7, "Step", 35)
myHanoiSSTower2.get_solutionDF()  # show stored solution when done

Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 3 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 4 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 2 from peg 3 to peg 1
Move disk 1 from peg 2 to peg 1
Move disk 3 from peg 3 to peg 2
Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 5 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 3 from peg 2 to peg 1
Move disk 1 from peg 3 to peg 2
Move disk 2 from peg 3 to peg 1
Move disk 1 from peg 2 to peg 1
Move disk 4 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 3 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move dis

,disk,fromPeg,toPeg
0,1,1,3
1,2,1,2
2,1,3,2
3,3,1,3
4,1,2,1
5,2,2,3
6,1,1,3
7,4,1,2
8,1,3,2
9,2,3,1


In [22]:
# show changes to stored values:
print(myHanoiSSTower2.divChars)
print(myHanoiSSTower2.dsks)
print(myHanoiSSTower2.moveCount)
print(myHanoiSSTower2.moveListDefault)

35
7
127
Step


<a id="ooDesign" name="ooDesign"></a>
### OO Design Considerations
Theoretically, as simulations grow larger, it may be desirable to have versions of the code that store the results versus versions that do not (so as not to expend the memory storing the steps when the resulting DF is not needed).  Python does support multi-inheritance, and so in theory, the objects could have followed an inheritance scheme like this:

<pre>
Base Object: output # of moves in solution 
             => Child that can output all disk moves (as simple steplist)
                => Child that can add more visual output to move list
             => Child that can store all disk moves in DF
                    => multi-inherence: child that can do all output + store results in DF
</pre>

Instead, a simpler design that avoids multi-inheritance was selected.  Multi-inheritance increases the complexity of maintenance and creates code that is harder to read and instantly see what it does.  There are many use cases for which this complexity is worth what it gains you, but not in this such a design feels over-engineered.  The final object model selected has just two "hanoi solution" objects in it:

<pre>
Base Object: can print out whatever we wish to see of the solution
             => Child Object: inherits all print options and stores the moves in a DF
</pre>

<a id="trace" name="trace"></a>
## Making The Solution Traceable (Watching The Recursion)
This modification to the solution is designed for purely academic reasons.  One of the reasons "The Towers of Hanoi" problem is so popular in code language education programs is that it is a problem best solved through recursion.  In fact, it is said that the problem is difficult to solve without recursion.  The purpose of the coding modifications in this section are to add tracer lines into the output that make visible the method calls and recursive method calls in action.

Output gets messy, but is interesting from a purely academic and educational standpoint.

In [23]:
class HanoiStoredSolutionTron(HanoiStoredSolution):
    ''' HanoiStoredSolutionTron -->\n\nAdds TRON (tracer on) functionality to HanoiStoredSolution.
Created as an illustration of the flow of recursive function calls.'''
    
    def __init__(self, numDisks, moveList="Stored", divider=30):
        print(HanoiStoredSolutionTron.__mro__)
        print("calling: __init__(self, " + str(numDisks) + ", " + str(moveList) + ", " + str(divider) + ")")
        HanoiStoredSolution.__init__(self, numDisks, moveList, divider)

    def _moveDisks(self, nDisks, source, target, auxiliary, moveList):
        print("calling: _moveDisks(self, " + str(nDisks) + ", " + str(source) + ", " + 
              str(target) + ", "+ str(auxiliary) + ", " + str(moveList) + ")")
        HanoiStoredSolution._moveDisks(self, nDisks, source, target, auxiliary, moveList)
        
    def _diskMovementProgression(self, nDisks, source, target, moveList="Visual"):
        print("calling: _diskMovementProgression(self, " + str(nDisks) + ", " + str(source) + 
              ", " + str(target) + ", " + str(moveList) + ")")
        HanoiStoredSolution._diskMovementProgression(self, nDisks, source, target, moveList)

    def _store_diskProgress(self, dsk, sourceID, targetID):
        print("calling: _store_diskProgress(self, " + str(dsk) + ", " + str(sourceID) + 
              ", " + str(targetID) + ")")
        HanoiStoredSolution._store_diskProgress(self, dsk, sourceID, targetID)
        
    def _output_diskProgress(self, nDisks, source, target, moveList):
        print("calling: _output_diskProgress(self, " + str(nDisks) + ", " + str(source) + ", " + 
              str(target) + ", " + str(moveList) + ")")
        HanoiStoredSolution._output_diskProgress(self, nDisks, source, target, moveList)
        
    def reInitialize(self, nDisks, moveList, divider=30):
        print("calling: reInitialize(self, " + str(nDisks) + ", " + str(moveList) + ", " + str(divider) + ")")
        HanoiStoredSolution.reInitialize(self, nDisks, moveList, divider)
        
    def __str__(self):
        print("calling: __str__(self)")
        return HanoiStoredSolution.__str__(self)
        
    def get_solutionDF(self):
        print("calling: get_solutionDF(self)")
        return HanoiStoredSolution.get_solutionDF(self)

print("HanoiStoredSolutionTron Object Loaded.")
print(HanoiStoredSolutionTron.__doc__)

HanoiStoredSolutionTron Object Loaded.
 HanoiStoredSolutionTron -->

Adds TRON (tracer on) functionality to HanoiStoredSolution.
Created as an illustration of the flow of recursive function calls.


In [24]:
hsst1 = HanoiStoredSolutionTron(3, "Visual", 72)  # function calls w/ full output showing

(<class '__main__.HanoiStoredSolutionTron'>, <class '__main__.HanoiStoredSolution'>, <class '__main__.HanoiSolution'>, <class 'object'>)
calling: __init__(self, 3, Visual, 72)
calling: _moveDisks(self, 3, ([3, 2, 1], 1), ([], 3), ([], 2), Visual)
calling: _moveDisks(self, 2, ([3, 2, 1], 1), ([], 2), ([], 3), Visual)
calling: _moveDisks(self, 1, ([3, 2, 1], 1), ([], 3), ([], 2), Visual)
calling: _moveDisks(self, 0, ([3, 2, 1], 1), ([], 2), ([], 3), Visual)
calling: _diskMovementProgression(self, 1, ([3, 2, 1], 1), ([], 3), Visual)
calling: _output_diskProgress(self, 1, ([3, 2, 1], 1), ([], 3), Visual)
Initial State:
------------------------------------------------------------------------
[3, 2, 1]
[]
[]
########################################################################
calling: _store_diskProgress(self, 1, 1, 3)
calling: _diskMovementProgression(self, 1, ([3, 2], 1), ([1], 3), Visual)
calling: _output_diskProgress(self, 1, ([3, 2], 1), ([1], 3), Visual)
Step 1:
Move disk 1 from pe

In [25]:
hsst1.reInitialize(3, "Count")  # Just function call trace and final move count

calling: reInitialize(self, 3, Count, 30)
(<class '__main__.HanoiStoredSolutionTron'>, <class '__main__.HanoiStoredSolution'>, <class '__main__.HanoiSolution'>, <class 'object'>)
calling: __init__(self, 3, Count, 30)
calling: _moveDisks(self, 3, ([3, 2, 1], 1), ([], 3), ([], 2), Count)
calling: _moveDisks(self, 2, ([3, 2, 1], 1), ([], 2), ([], 3), Count)
calling: _moveDisks(self, 1, ([3, 2, 1], 1), ([], 3), ([], 2), Count)
calling: _moveDisks(self, 0, ([3, 2, 1], 1), ([], 2), ([], 3), Count)
calling: _diskMovementProgression(self, 1, ([3, 2, 1], 1), ([], 3), Count)
calling: _output_diskProgress(self, 1, ([3, 2, 1], 1), ([], 3), Count)
calling: _store_diskProgress(self, 1, 1, 3)
calling: _diskMovementProgression(self, 1, ([3, 2], 1), ([1], 3), Count)
calling: _output_diskProgress(self, 1, ([3, 2], 1), ([1], 3), Count)
calling: _store_diskProgress(self, 1, 1, 3)
calling: _moveDisks(self, 0, ([], 2), ([1], 3), ([3, 2], 1), Count)
calling: _diskMovementProgression(self, 2, ([3], 1), ([2], 

In [26]:
hsst1.get_solutionDF()

calling: get_solutionDF(self)


,disk,fromPeg,toPeg
0,1,1,3
1,2,1,2
2,1,3,2
3,3,1,3
4,1,2,1
5,2,2,3
6,1,1,3


<a id="Research" name="Research"></a>
## Hannoi Solutions Research and Experimentation
Code presented here, when it has a source, the source is sited.  Then edits and enhancements are made to this code experimenting with it in different ways as part of the research that ultimately led to the solution given at the start of this notebook.

In [27]:
# Example 1:
# source: http://www.python-course.eu/towers_of_hanoi.php

''' This code solves the puzzle, but shows us nothing in terms of how it does it.
    What we really want is a program that solves the puzzle and provides a solution. 
    But this code is a good clean example of recursive programming.
'''

def hanoi(n, source, helper, target):
    if n > 0:
        # move tower of size n - 1 to helper:
        hanoi(n - 1, source, target, helper)
        # move disk from source peg to target peg
        if source:
            target.append(source.pop())
        # move tower of size n-1 from helper to target
        hanoi(n - 1, helper, source, target)
        
source = [4,3,2,1]
target = []
helper = []
hanoi(len(source),source,helper,target)

print(source, helper, target)
  # modified from source for Python 2.7 as well as 3.x compatibility

[] [] [4, 3, 2, 1]


In [28]:
# source: http://www.python-course.eu/towers_of_hanoi.php

''' This is better, but the solution provided is output in such a mess that its hard to
    see the solution from what is essentially a trace of the inner workings of the program.
    This code makes a good demonstration of how the recursive algorithm does its work though.
'''

def hanoi(n, source, helper, target):
    print("hanoi( " + str(n) + str(source) + str(helper) + str(target) + " called")
      # modified from source for 2.7 and 3.x compatibility
        
    if n > 0:
        # move tower of size n - 1 to helper:
        hanoi(n - 1, source, target, helper)
        # move disk from source peg to target peg
        if source[0]:
            disk = source[0].pop()
            print("moving " + str(disk) + " from " + source[1] + " to " + target[1])
               # modified from source for Python 2.7 and 3.x compatibility
            target[0].append(disk)
        # move tower of size n-1 from helper to target
        hanoi(n - 1, helper, source, target)
        
source = ([4,3,2,1], "source")
target = ([], "target")
helper = ([], "helper")
hanoi(len(source[0]),source,helper,target)

print(source, helper, target)
  # modified from source for Python 2.7 as well as 3.x compatibility

hanoi( 4([4, 3, 2, 1], 'source')([], 'helper')([], 'target') called
hanoi( 3([4, 3, 2, 1], 'source')([], 'target')([], 'helper') called
hanoi( 2([4, 3, 2, 1], 'source')([], 'helper')([], 'target') called
hanoi( 1([4, 3, 2, 1], 'source')([], 'target')([], 'helper') called
hanoi( 0([4, 3, 2, 1], 'source')([], 'helper')([], 'target') called
moving 1 from source to helper
hanoi( 0([], 'target')([4, 3, 2], 'source')([1], 'helper') called
moving 2 from source to target
hanoi( 1([1], 'helper')([4, 3], 'source')([2], 'target') called
hanoi( 0([1], 'helper')([2], 'target')([4, 3], 'source') called
moving 1 from helper to target
hanoi( 0([4, 3], 'source')([], 'helper')([2, 1], 'target') called
moving 3 from source to helper
hanoi( 2([2, 1], 'target')([4], 'source')([3], 'helper') called
hanoi( 1([2, 1], 'target')([3], 'helper')([4], 'source') called
hanoi( 0([2, 1], 'target')([4], 'source')([3], 'helper') called
moving 1 from target to source
hanoi( 0([3], 'helper')([2], 'target')([4, 1], 'sourc

In [29]:
# let's take the previous code and modify it so we can run w/ and w/o the trace for better analysis
# some other tweaks to language and output will also be made

def hanoi(n, source, helper, target, tron = False, diskTrace = False):
    if tron == True:     # tron = "Tracer On" and was the title of a popular movie set in a virtual world
        print("hanoi( " + str(n) + str(source) + str(helper) + str(target) + " called")
            # modified from source for compatibility with Python 2.7 or 3.x        
        
    if n > 0:
        # move tower of size n - 1 to helper:
        hanoi(n - 1, source, target, helper, tron, diskTrace)
        # move disk from source peg to target peg
        if source[0]:
            disk = source[0].pop()
            if diskTrace == True:
                mv = "move disk " + str(disk)
            else:
                mv = "move"
            print(mv + " from " + source[1] + " to " + target[1])
            target[0].append(disk)
        # move tower of size n-1 from helper to target
        hanoi(n - 1, helper, source, target, tron, diskTrace)

# set up pegs        
source = ([4,3,2,1], "source")
target = ([], "target")
helper = ([], "helper")

# run simulation and print results:
print(source + helper + target)
hanoi(len(source[0]),source,helper,target, diskTrace = True)  
       # add final argument of True to turn full program trace back on
       # then output will look like previous cell
       # it is disabled here to demonstrate the cleaner "solution" output
print(source + helper + target)
   # these lines modified from source for 2.7 and 3.x compatibility

([4, 3, 2, 1], 'source', [], 'helper', [], 'target')
move disk 1 from source to helper
move disk 2 from source to target
move disk 1 from helper to target
move disk 3 from source to helper
move disk 1 from target to source
move disk 2 from target to helper
move disk 1 from source to helper
move disk 4 from source to target
move disk 1 from helper to target
move disk 2 from helper to source
move disk 1 from target to source
move disk 3 from helper to target
move disk 1 from source to helper
move disk 2 from source to target
move disk 1 from helper to target
([], 'source', [], 'helper', [4, 3, 2, 1], 'target')


In [31]:
# source: http://www.python-course.eu/towers_of_hanoi.php
# this code used as starting point and then modified and enhanced considerably to create this version        

# this alteration to the source will make the code a bit more self contained and will give options for which
# peg gets moved to which peg.  For simplicity, the story it tells is we are moving from "peg 1" to
# "peg 3" (labeled simply 1, 2, 3) rather than "source", "target", etc.
# user can chose which of the 3 pegs is source, target, and what earlier code called "helper" or "auxilliary"

def hanoi(n, start=1, end=3, spare=2, tron=False, diskTrace=False):
    # sets up data structure(s) to pass into our recursive child function
    
    if sorted([start, end, spare]) != [1,2,3]:
        raise ValueError("Arguments: start, end, spare - must be unique and can only contain the values 1, 2, or 3.\n" +
                                     "This tells the program which peg (of the 3 pegs) is used for what role in the game.")
    
    hanoi_towers = [(list(range(n, 0, -1)), start), ([], end), ([], spare)]
    step_count = [0]

    # embedded child function does all the actual work:
                            #start  #spare  #end
    def hanoiRecurModule(n, source, helper, target, tron = False, diskTrace = False):
        if tron == True:     # tron = "Tracer On" and was the title of a popular movie set in a virtual world
            print("hanoiRecurModule( " + str(n) + str(source) + str(helper) + str(target) + " called")
                # modified from source for compatibility with Python 2.7 or 3.x        

        if n > 0:
            # move tower of size n - 1 to helper:
            hanoiRecurModule(n - 1, source, target, helper, tron, diskTrace)
            # move disk from source peg to target peg
            if source[0]:
                disk = source[0].pop()
                if diskTrace == True:
                    mv = "move disk " + str(disk)
                else:
                    mv = "move"
                print(mv + " from " + str(source[1]) + " to " + str(target[1]))
                step_count[0] += 1
                target[0].append(disk)
            # move tower of size n-1 from helper to target
            hanoiRecurModule(n - 1, helper, source, target, tron, diskTrace)
                        #start           #spare           #end            
    hanoiRecurModule(n, hanoi_towers[0], hanoi_towers[2], hanoi_towers[1], tron, diskTrace) 
    if step_count == [1]:
        endSentence = " step."
    else:
        endSentence = " steps."
    print("Task completed in " + str(step_count)[1:-1] + endSentence)
        
# run simulation and print results: 
hanoi(4, start=1, end=3, spare=2, diskTrace = True)

move disk 1 from 1 to 2
move disk 2 from 1 to 3
move disk 1 from 2 to 3
move disk 3 from 1 to 2
move disk 1 from 3 to 1
move disk 2 from 3 to 2
move disk 1 from 1 to 2
move disk 4 from 1 to 3
move disk 1 from 2 to 3
move disk 2 from 2 to 1
move disk 1 from 3 to 1
move disk 3 from 2 to 3
move disk 1 from 1 to 2
move disk 2 from 1 to 3
move disk 1 from 2 to 3
Task completed in 15 steps.


In [32]:
hanoi(1, start=1, end=3, spare=2, diskTrace = True)

move disk 1 from 1 to 3
Task completed in 1 step.


In [33]:
hanoi(1, start=1, end=3, spare=2, diskTrace = False)

move from 1 to 3
Task completed in 1 step.


In [34]:
# testing the ValueError
try:
    hanoi(4, start=1, end=3, spare=1, diskTrace = True)
except Exception as ee:
    print(str(type(ee))+": \n"+str(ee))

<class 'ValueError'>: 
Arguments: start, end, spare - must be unique and can only contain the values 1, 2, or 3.
This tells the program which peg (of the 3 pegs) is used for what role in the game.


In [35]:
# with tracer on
hanoi(3, start=1, end=3, spare=2, tron=True, diskTrace = True)

hanoiRecurModule( 3([3, 2, 1], 1)([], 2)([], 3) called
hanoiRecurModule( 2([3, 2, 1], 1)([], 3)([], 2) called
hanoiRecurModule( 1([3, 2, 1], 1)([], 2)([], 3) called
hanoiRecurModule( 0([3, 2, 1], 1)([], 3)([], 2) called
move disk 1 from 1 to 3
hanoiRecurModule( 0([], 2)([3, 2], 1)([1], 3) called
move disk 2 from 1 to 2
hanoiRecurModule( 1([1], 3)([3], 1)([2], 2) called
hanoiRecurModule( 0([1], 3)([2], 2)([3], 1) called
move disk 1 from 3 to 2
hanoiRecurModule( 0([3], 1)([], 3)([2, 1], 2) called
move disk 3 from 1 to 3
hanoiRecurModule( 2([2, 1], 2)([], 1)([3], 3) called
hanoiRecurModule( 1([2, 1], 2)([3], 3)([], 1) called
hanoiRecurModule( 0([2, 1], 2)([], 1)([3], 3) called
move disk 1 from 2 to 1
hanoiRecurModule( 0([3], 3)([2], 2)([1], 1) called
move disk 2 from 2 to 3
hanoiRecurModule( 1([1], 1)([], 2)([3, 2], 3) called
hanoiRecurModule( 0([1], 1)([3, 2], 3)([], 2) called
move disk 1 from 1 to 3
hanoiRecurModule( 0([], 2)([], 1)([3, 2, 1], 3) called
Task completed in 7 steps.


In [36]:
# source: https://en.wikipedia.org/wiki/Tower_of_Hanoi
#         recursive implementation section

# this solution requires slightly more steps than the above code, but is still quite elegant
# it also provides the best visual metaphor for the solution in its output of any of the
# code in this notebook yet

A = [5,4,3,2,1]
B = []
C = []

def move(n, source, target, auxiliary):
    if n > 0:
        # move n-1 disks from source to auxiliary, so they are out of the way
        move(n-1, source, auxiliary, target)

        # move the nth disk from source to target
        target.append(source.pop())

        # Display our progress
        print(str(A) + '\n' + str(B) + '\n' + str(C) + '\n' + '##############')
          # modified from source so it will work in Python 2.7 or Python 3.x

        # move the n-1 disks that we left on auxiliary onto target
        move(n-1, auxiliary, target, source)

# initiate call from source A to target C with auxiliary B
move(5, A, C, B)

[5, 4, 3, 2]
[]
[1]
##############
[5, 4, 3]
[2]
[1]
##############
[5, 4, 3]
[2, 1]
[]
##############
[5, 4]
[2, 1]
[3]
##############
[5, 4, 1]
[2]
[3]
##############
[5, 4, 1]
[]
[3, 2]
##############
[5, 4]
[]
[3, 2, 1]
##############
[5]
[4]
[3, 2, 1]
##############
[5]
[4, 1]
[3, 2]
##############
[5, 2]
[4, 1]
[3]
##############
[5, 2, 1]
[4]
[3]
##############
[5, 2, 1]
[4, 3]
[]
##############
[5, 2]
[4, 3]
[1]
##############
[5]
[4, 3, 2]
[1]
##############
[5]
[4, 3, 2, 1]
[]
##############
[]
[4, 3, 2, 1]
[5]
##############
[1]
[4, 3, 2]
[5]
##############
[1]
[4, 3]
[5, 2]
##############
[]
[4, 3]
[5, 2, 1]
##############
[3]
[4]
[5, 2, 1]
##############
[3]
[4, 1]
[5, 2]
##############
[3, 2]
[4, 1]
[5]
##############
[3, 2, 1]
[4]
[5]
##############
[3, 2, 1]
[]
[5, 4]
##############
[3, 2]
[]
[5, 4, 1]
##############
[3]
[2]
[5, 4, 1]
##############
[3]
[2, 1]
[5, 4]
##############
[]
[2, 1]
[5, 4, 3]
##############
[1]
[2]
[5, 4, 3]
##############
[1]
[]
[5, 4, 3, 2]
#

In [38]:
# Solution Experiment One
# modified from code presented in previous cells ..

# why are we asking the user for things the code can do for us ...
# this version is more self-contained and requires less of the user to run it

def solveHanoi(numDisks):
    peg1 = list(range(numDisks, 0, -1))
    peg2 = []
    peg3 = []
    
    def moveDisks(numDisks, source, target, auxiliary):
        # python allows nested functions but this may not be best practice
        #        completing the code this way just as an experiment
        
        if numDisks > 0:
            # move n-1 disks from source to auxiliary, so they are out of the way
            moveDisks(numDisks-1, source, auxiliary, target)

            # move the nth disk from source to target
            target.append(source.pop())

            # Display our progress
            print(str(peg1) + '\n' + str(peg2) + '\n' + str(peg3) + '\n' + '##############')
            # modified from source so it will work in Python 2.7 or Python 3.x

            # move the n-1 disks that we left on auxiliary onto target
            moveDisks(numDisks-1, auxiliary, target, source)
            
    moveDisks(numDisks, source=peg1, target=peg3, auxiliary=peg2)

# initiate call from source A to target C with auxiliary B
solveHanoi(5)

[5, 4, 3, 2]
[]
[1]
##############
[5, 4, 3]
[2]
[1]
##############
[5, 4, 3]
[2, 1]
[]
##############
[5, 4]
[2, 1]
[3]
##############
[5, 4, 1]
[2]
[3]
##############
[5, 4, 1]
[]
[3, 2]
##############
[5, 4]
[]
[3, 2, 1]
##############
[5]
[4]
[3, 2, 1]
##############
[5]
[4, 1]
[3, 2]
##############
[5, 2]
[4, 1]
[3]
##############
[5, 2, 1]
[4]
[3]
##############
[5, 2, 1]
[4, 3]
[]
##############
[5, 2]
[4, 3]
[1]
##############
[5]
[4, 3, 2]
[1]
##############
[5]
[4, 3, 2, 1]
[]
##############
[]
[4, 3, 2, 1]
[5]
##############
[1]
[4, 3, 2]
[5]
##############
[1]
[4, 3]
[5, 2]
##############
[]
[4, 3]
[5, 2, 1]
##############
[3]
[4]
[5, 2, 1]
##############
[3]
[4, 1]
[5, 2]
##############
[3, 2]
[4, 1]
[5]
##############
[3, 2, 1]
[4]
[5]
##############
[3, 2, 1]
[]
[5, 4]
##############
[3, 2]
[]
[5, 4, 1]
##############
[3]
[2]
[5, 4, 1]
##############
[3]
[2, 1]
[5, 4]
##############
[]
[2, 1]
[5, 4, 3]
##############
[1]
[2]
[5, 4, 3]
##############
[1]
[]
[5, 4, 3, 2]
#

In [39]:
### Verson One:  Object code
## Useful help topic: http://stackoverflow.com/questions/3277367/how-does-pythons-super-work-with-multiple-inheritance

import pandas as pd
import numpy as np

class HanoiSolution_v1(object):
    def __init__(self, numDisks):
        
        # peg structure: ( [ disks ], Peg_ID_Number )
        self._peg1 = (list(range(numDisks, 0, -1)), 1)
        self._peg2 = ([], 2)
        self._peg3 = ([], 3)
        
        self.dsks = numDisks  # number of disks for simulation
        self.moveCount = 0    # move counter
        self.divChars = 25    # number of characters for divider used in output
        
        self._solutionPD = pd.DataFrame({'disk':[],'fromPeg':[], 'toPeg':[]}, dtype=np.int64)
        self._moveDisks(nDisks=numDisks, source=self._peg1, target=self._peg3, auxiliary=self._peg2)
            
    def _moveDisks(self, nDisks, source, target, auxiliary):        
        if nDisks > 0:           
            # move n-1 disks from source to auxiliary, so they are out of the way
            self._moveDisks(nDisks-1, source, auxiliary, target)

            # move the nth disk from source to target
            target[0].append(source[0].pop())

            # Display our progress (create each step of the answer and output it)
            self.moveCount += 1
            print("Step %d:" %self.moveCount)
            print("-"*self.divChars)
            print("Move disk " + str(nDisks) + " from " + str(source[1]) + " to " + str(target[1]))
            print(str(self._peg1[0]) + '\n' + str(self._peg2[0]) + '\n' + str(self._peg3[0]) + 
                  '\n' + '#'*self.divChars)
            self._store_diskProgress(nDisks, source[1], target[1])

            # move the n-1 disks that were left on auxiliary to target
            self._moveDisks(nDisks-1, auxiliary, target, source)
                
        return self.moveCount
    
    def _store_diskProgress(self, dsk, sourceID, targetID):
        # Builds this: self._solutionPD = pd.DataFrame({ 'disk':[],'fromPeg':[], 'toPeg':[] })
        self._solutionPD = self._solutionPD.append(pd.DataFrame({ 'disk':[dsk],'fromPeg':[sourceID], 
                                                   'toPeg':[targetID] }), ignore_index=True)
    
    def __str__(self):
        return "%d disks would take %d moves to solve." %(self.dsks, self.moveCount)

myHanoiTower_v1 = HanoiSolution_v1(5)
print(myHanoiTower_v1)

Step 1:
-------------------------
Move disk 1 from 1 to 3
[5, 4, 3, 2]
[]
[1]
#########################
Step 2:
-------------------------
Move disk 2 from 1 to 2
[5, 4, 3]
[2]
[1]
#########################
Step 3:
-------------------------
Move disk 1 from 3 to 2
[5, 4, 3]
[2, 1]
[]
#########################
Step 4:
-------------------------
Move disk 3 from 1 to 3
[5, 4]
[2, 1]
[3]
#########################
Step 5:
-------------------------
Move disk 1 from 2 to 1
[5, 4, 1]
[2]
[3]
#########################
Step 6:
-------------------------
Move disk 2 from 2 to 3
[5, 4, 1]
[]
[3, 2]
#########################
Step 7:
-------------------------
Move disk 1 from 1 to 3
[5, 4]
[]
[3, 2, 1]
#########################
Step 8:
-------------------------
Move disk 4 from 1 to 2
[5]
[4]
[3, 2, 1]
#########################
Step 9:
-------------------------
Move disk 1 from 3 to 2
[5]
[4, 1]
[3, 2]
#########################
Step 10:
-------------------------
Move disk 2 from 3 to 1
[5, 2]
[4, 1]
[

In [40]:
# exploration of the object structure:
print(type(myHanoiTower_v1))

<class '__main__.HanoiSolution_v1'>


In [41]:
myHanoiTower_v1._solutionPD

,disk,fromPeg,toPeg
0,1,1,3
1,2,1,2
2,1,3,2
3,3,1,3
4,1,2,1
5,2,2,3
6,1,1,3
7,4,1,2
8,1,3,2
9,2,3,1
